# Reduction of 1.3m images using IRAF

We demonstrate the reduction of images obtained through the 2k CCD installed at ARIES Devasthal Observatory. The reduction will be performed using standard IRAF tasks called through PyRAF. (06/02/2020)

# 1. Importing the packages

In [5]:
from pyraf import iraf
from iraf import noao,imred,specred
from stsci import tools
import numpy as np
import os
import sys

In [ ]:
os.system("ls *bias* > bias.in")
iraf.imcombine('@bias.in', 'mbias', combine="median",rdnoise=7.5,gain=2.0)
os.system("ls flat* j0644* > subbias.in")
os.system("sed s/.fits/b.fits/g subbias.in > subbias.out")
iraf.imarith('@subbias.in','-','mbias.fits','@subbias.out')
print("***********************************************************")
print("Bias correction done!")


Feb  6 15:29: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
            bias_1.fits
            bias_2.fits
            bias_3.fits
            bias_4.fits
            bias_5.fits

  Output image = mbias, ncombine = 5


In [ ]:
os.system("ls flat*.fits > flat.in")
iraf.imcombine('@flat.in', 'mflat.fits', combine="median",rdnoise=7.5,gain=2.0)

#making master flat.
iraf.response("mflat","mflat", "nmflat", interactive="no", threshold="INDEF", sample="*", naverage=1,
      function="spline3", order=20, low_reject=0., high_reject=0., niterate=1,
      grow=0., graphics="stdgraph", cursor="")

#iraf.imstat('nmflat.fits')    
os.system("ls J105829*b.fits GD248*b.fits > flatf.in")
os.system("sed s/b.fits/bf.fits/g flatf.in > flatf.out")
iraf.imarith('@flatf.in','/','nmflat.fits','@flatf.out')
print("***********************************************************")
print("Flat correction done!")


# For cosmic ray removal, we will use Lacosmic.

In [ ]:
# Removal of cosmic rays from the images
array, header=cosmics.fromfits("J105829_250_3500_4_Grism_7_2015_06_17_yf170018bf.fits")
c= cosmics.cosmicsimage(array, gain=1.22, readnoise=4.8, sigclip=4.5, sigfrac=0.3, objlim=4.5)
c.run(maxiter=4)
cosmics.tofits("J105829_250_3500_4_Grism_7_2015_06_17_yf170018bfc.fits", c.cleanarray, header)

array, header=cosmics.fromfits("GD248_standard_250_3500_4_Grism_7_yf170031bf.fits")
c= cosmics.cosmicsimage(array, gain=1.22, readnoise=4.8, sigclip=4.5, sigfrac=0.3, objlim=4.5)
c.run(maxiter=4)
cosmics.tofits("GD248_standard_250_3500_4_Grism_7_yf170031bfc.fits", c.cleanarray, header)

print("***********************************************************")